# Pełny Cykl Życia Modelu Sentymentu

Ten notebook krok po kroku demonstruje cały cykl życia modelu do klasyfikacji sentymentu, od przygotowania danych, przez eksperymenty i wybór najlepszego modelu, aż po jego użycie do predykcji na nowych danych.

## Sekcja 1: Przygotowanie i Konfiguracja

Zakładamy, że dane treningowe zostały już przygotowane i znajdują się w pliku `artifacts/data/all_train.csv`. W tej komórce definiujemy zmienne, których będziemy używać w całym procesie.

In [ ]:
import os
import pandas as pd
import json

os.environ['EXPERIMENT_ID'] = "exp_notebook_tutorial"
os.environ['INPUT_DATA'] = "artifacts/data/all_train.csv"

## Sekcja 2: Generowanie Podziałów Danych

Tworzymy podziały danych dla naszego eksperymentu. Użyjemy strategii `train-val` z wydzielonym zbiorem testowym (20% danych), który posłuży do ostatecznej oceny najlepszego modelu.

In [ ]:
!uv run cli generate-splits --experiment-id $EXPERIMENT_ID --input-file $INPUT_DATA --test-size 0.2 --backtesting-strategy train-val

## Sekcja 3: Uruchomienie Eksperymentów i Porównanie Modeli

Teraz uruchomimy proces treningu i walidacji dla każdego z wybranych modeli. Po każdym przebiegu wyświetlimy zaktualizowaną tabelę z porównaniem wyników.

### Model: svm-base

In [ ]:
!uv run cli run-backtesting --experiment-id $EXPERIMENT_ID --model-config-name svm-base

In [ ]:
!uv run cli compare-models --experiment-id $EXPERIMENT_ID --output-file artifacts/experiments/$EXPERIMENT_ID/comparison.csv --run-type backtesting
comparison_df = pd.read_csv(f"artifacts/experiments/{os.environ['EXPERIMENT_ID']}/comparison.csv")
display(comparison_df)

### Model: svm-opt

In [ ]:
!uv run cli run-backtesting --experiment-id $EXPERIMENT_ID --model-config-name svm-opt

In [ ]:
!uv run cli compare-models --experiment-id $EXPERIMENT_ID --output-file artifacts/experiments/$EXPERIMENT_ID/comparison.csv --run-type backtesting
comparison_df = pd.read_csv(f"artifacts/experiments/{os.environ['EXPERIMENT_ID']}/comparison.csv")
display(comparison_df)

### Model: Gemma without tuning (prompt-gemma)

In [ ]:
!uv run cli run-backtesting --experiment-id $EXPERIMENT_ID --model-config-name prompt-gemma-3-1b-it

### Model: bert-micro-base

In [ ]:
!uv run cli run-backtesting --experiment-id $EXPERIMENT_ID --model-config-name bert-micro-base

In [ ]:
!uv run cli compare-models --experiment-id $EXPERIMENT_ID --output-file artifacts/experiments/$EXPERIMENT_ID/comparison.csv --run-type backtesting
comparison_df = pd.read_csv(f"artifacts/experiments/{os.environ['EXPERIMENT_ID']}/comparison.csv")
display(comparison_df)

In [ ]:
!uv run cli compare-models --experiment-id $EXPERIMENT_ID --output-file artifacts/experiments/$EXPERIMENT_ID/comparison.csv --run-type backtesting
comparison_df = pd.read_csv(f"artifacts/experiments/{os.environ['EXPERIMENT_ID']}/comparison.csv")
display(comparison_df)

### Model: bert-micro-long

In [ ]:
!uv run cli run-backtesting --experiment-id $EXPERIMENT_ID --model-config-name bert-micro-long

In [ ]:
!uv run cli compare-models --experiment-id $EXPERIMENT_ID --output-file artifacts/experiments/$EXPERIMENT_ID/comparison.csv --run-type backtesting
comparison_df = pd.read_csv(f"artifacts/experiments/{os.environ['EXPERIMENT_ID']}/comparison.csv")
display(comparison_df)

### Model: roberta-base

In [ ]:
!uv run cli run-backtesting --experiment-id $EXPERIMENT_ID --model-config-name roberta-base

In [ ]:
!uv run cli compare-models --experiment-id $EXPERIMENT_ID --output-file artifacts/experiments/$EXPERIMENT_ID/comparison.csv --run-type backtesting
comparison_df = pd.read_csv(f"artifacts/experiments/{os.environ['EXPERIMENT_ID']}/comparison.csv")
display(comparison_df)

### Eksperyment: Zwiększenie liczby epok dla `roberta-base`

Tworzymy nową konfigurację `roberta-base-3` w locie, zmieniając liczbę epok treningowych, aby sprawdzić, czy dłuższy trening poprawi wynik.

In [ ]:
# Wczytanie bazowej konfiguracji
with open('model_configs/roberta-base.json', 'r') as f:
    config = json.load(f)

# Modyfikacja hiperparametru
config["training_arguments"]["num_train_epochs"] = 3

# Zapis nowej konfiguracji
with open('model_configs/roberta-base-3.json', 'w') as f:
    json.dump(config, f, indent=4)

print("Utworzono nową konfigurację: model_configs/roberta-base-3.json")

In [ ]:
# Uruchomienie eksperymentu dla nowej konfiguracji
!uv run cli run-backtesting --experiment-id $EXPERIMENT_ID --model-config-name roberta-base-3

In [ ]:
# Ostateczne porównanie
!uv run cli compare-models --experiment-id $EXPERIMENT_ID --output-file artifacts/experiments/$EXPERIMENT_ID/comparison.csv --run-type backtesting
comparison_df = pd.read_csv(f"artifacts/experiments/{os.environ['EXPERIMENT_ID']}/comparison.csv")
display(comparison_df)

## Sekcja 4: Wybór Najlepszego Modelu

Na podstawie powyższych wyników, jako najlepszy model do dalszych kroków wybieramy `roberta-base-3` (założenie na potrzeby tego tutoriala).

In [ ]:
os.environ['BEST_MODEL'] = "roberta-base"

## Sekcja 5: Oszacowanie Wydajności na Zbiorze Testowym

Wybrany model sprawdzamy na odłożonym wcześniej zbiorze testowym. To da nam ostateczną, bezstronną ocenę jego jakości.

In [ ]:
!uv run cli estimate-performance --experiment-id $EXPERIMENT_ID --model-config-name $BEST_MODEL

## Sekcja 6: Trening Finalnego Modelu

Mając pewność co do jakości naszego modelu, trenujemy go po raz ostatni na pełnym zbiorze danych (trening + walidacja), aby był gotowy do użycia produkcyjnego.

In [ ]:
!uv run cli train-final-model --experiment-id $EXPERIMENT_ID --model-config-name $BEST_MODEL --model-output-dir artifacts/trained_models/

## Sekcja 7: Predykcja na Nowych Danych

Symulujemy scenariusz produkcyjny: pojawiły się nowe dane i chcemy poznać ich sentyment. Użyjemy do tego naszego generatora danych.

In [ ]:
!uv run cli data-generate --name new_unseen_data --examples-per-sentiment 5 --lang pl
os.environ['NEW_DATA_FILE'] = "artifacts/data/new_data/new_unseen_data.csv"

Używamy naszego finalnego modelu do wykonania predykcji na nowych danych.

In [ ]:
FINAL_MODEL_PATH = f"artifacts/trained_models/{os.environ['EXPERIMENT_ID']}_final/{os.environ['BEST_MODEL']}"
PREDICTION_OUTPUT_FILE = f"artifacts/predictions/new_data_predictions.csv"

!uv run cli predict-new --model-path $FINAL_MODEL_PATH --input-file $NEW_DATA_FILE --output-file $PREDICTION_OUTPUT_FILE

# Wyświetlenie predykcji
predictions_df = pd.read_csv(PREDICTION_OUTPUT_FILE)
print("Wygenerowane predykcje:")
display(predictions_df)

## Sekcja 8: Symulacja Oceny Jakości

W realnym scenariuszu dla nowo wygenerowanych danych nie mielibyśmy prawdziwych etykiet. Aby jednak zademonstrować działanie skryptu `evaluate.py`, zasymulujemy ocenę, traktując sentyment, o który prosiliśmy generator, jako "prawdziwą etykietę".

In [ ]:
!python evaluate.py --predictions-file $PREDICTION_OUTPUT_FILE --ground-truth-file $NEW_DATA_FILE